
# Which user groups perform what action
To find which user segments (e.g., male vs female, age groups, hometowns) are more likely to perform certain actions (like posting content, buying, etc.), you can apply a mix of exploratory data analysis, statistical testing, and predictive modeling.

A comprehensive Python example walking through these steps using a synthetic dataset. It includes:
* Exploratory analysis (group-by & action rates)
* Statistical significance testing (Chi-square for gender & action)
* Predictive modeling (logistic regression + feature importance)
* Interpretation notes and comments at every step

## Imports & Sample Data Creation

In [20]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, ttest_ind, fisher_exact
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import statsmodels.api as sm

# Create a synthetic user dataset with demographics and binary action flag
np.random.seed(42)

n = 1000
df = pd.DataFrame({
    'user_id': range(1, n+1),
    'gender': np.random.choice(['Male', 'Female'], n, p=[0.5, 0.5]),
    'age': np.random.randint(18, 60, n),
    'hometown': np.random.choice(['TownA', 'TownB', 'TownC'], n, p=[0.4, 0.35, 0.25]),
})

# Create a binary outcome 'action_flag' influenced by gender and age
# (e.g., females and younger users slightly more likely to perform the action)
df['action_flag'] = np.random.binomial(
    1,
    p=0.1 + 0.1*(df['gender'] == 'Female') + 0.002*(60 - df['age']),
    size=n
)

# Show head of dataset
df.head()

,user_id,gender,age,hometown,action_flag
0,1,Male,29,TownB,0
1,2,Female,33,TownB,1
2,3,Female,41,TownA,0
3,4,Female,36,TownA,0
4,5,Male,25,TownC,0


## Exploratory Analysis: Action Rates by Segment

In [21]:
# Calculate action rate by gender
action_rate_gender = df.groupby('gender')['action_flag'].mean().reset_index()
print("Action Rate by Gender:\n", action_rate_gender)

# Calculate action rate by age groups (create bins)
df['age_group'] = pd.cut(df['age'], bins=[17, 25, 35, 45, 60], labels=['18-25','26-35','36-45','46-60'])
action_rate_age = df.groupby('age_group', observed=False)['action_flag'].mean().reset_index()
print("\nAction Rate by Age Group:\n", action_rate_age)

# Calculate action rate by hometown
action_rate_home = df.groupby('hometown')['action_flag'].mean().reset_index()
print("\nAction Rate by Hometown:\n", action_rate_home)

Action Rate by Gender:
    gender  action_flag
0  Female     0.265594
1    Male     0.139165

Action Rate by Age Group:
   age_group  action_flag
0     18-25     0.239583
1     26-35     0.219124
2     36-45     0.197248
3     46-60     0.171091

Action Rate by Hometown:
   hometown  action_flag
0    TownA     0.190000
1    TownB     0.213675
2    TownC     0.204819


In [22]:
df.head()

,user_id,gender,age,hometown,action_flag,age_group
0,1,Male,29,TownB,0,26-35
1,2,Female,33,TownB,1,26-35
2,3,Female,41,TownA,0,36-45
3,4,Female,36,TownA,0,36-45
4,5,Male,25,TownC,0,18-25


### Interpretation
This gives a first idea where action rates differ. For example, if females show a 20% action rate vs males 10%, that’s a strong signal.

##  Statistical Significance Testing: Gender vs Action

Since both variables are categorical (gender and binary action), **Chi-square test is the standard** approach to test independence.

In [23]:
# Create contingency table
contingency_table = pd.crosstab(df['gender'], df['action_flag'])
print("\nContingency Table (Gender vs Action):\n", contingency_table)

# Chi-square test of independence
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
print(f"\nChi-square test results:\nChi2 = {chi2:.3f}, p-value = {p_value:.5f}")

if p_value < 0.05:
    print("=> Reject null hypothesis: Gender and action are statistically dependent.")
else:
    print("=> Fail to reject null hypothesis: No significant dependency found.")


Contingency Table (Gender vs Action):
 action_flag    0    1
gender               
Female       365  132
Male         433   70

Chi-square test results:
Chi2 = 24.011, p-value = 0.00000
=> Reject null hypothesis: Gender and action are statistically dependent.


### When to use Chi-square, T-test, or Fisher’s Exact Test

| Situation                                     | Recommended Test      | Why?                                 |
|-----------------------------------------------|----------------------|-------------------------------------|
| Two categorical variables (large sample)      | Chi-square test      | Tests independence between categories |
| Binary categorical vs continuous (or interval) | T-test / ANOVA       | Compares means between two or more groups |
| Two categorical variables (small sample or low counts) | Fisher’s Exact Test | Exact test valid for small samples   |

#### Side Note: Contingency table
A contingency table summarizes the relationship between categorical variables by counting occurrences. It is fundamental in categorical data analysis.
* It summarizes data by **counting how many times different combinations of categories occur**.
* Typically, one categorical variable is displayed along the rows and another along the columns.
* Each cell in the table represents the count (or sometimes proportion) of observations that fall into that combination of categories.


In [24]:
import pandas as pd

# Example data
data = {
    'Gender': ['Male', 'Female', 'Male', 'Female', 'Male'],
    'Fruit': ['Apple', 'Banana', 'Apple', 'Orange', 'Banana']
}

df_cont = pd.DataFrame(data)

# Create a contingency table
contingency = pd.crosstab(df_cont['Gender'], df_cont['Fruit'])
print(contingency)

Fruit   Apple  Banana  Orange
Gender                       
Female      0       1       1
Male        2       1       0


## Predictive Modeling: Logistic Regression (using statsmodels)
We model the probability of the action based on gender, age, and hometown. This gives effect sizes and feature importance.

Note: We use statsmodels here, because scikit-learn is excellent for predictive modeling and production ML pipelines. But here we are interested in finding which segments (gender, age, etc.) are more likely to take an action — **it’s not just about prediction accuracy**, it’s about:

1.	Coefficient estimates with statistical meaning
- statsmodels outputs p-values, confidence intervals, and standard errors for each coefficient.
- This lets you answer: **“Is this variable’s effect statistically significant?” and “How certain is the magnitude?”**
- (scikit-learn’s LogisticRegression doesn’t provide this directly — it focuses on prediction)

2. Clear mapping between variables and coefficients
- The .summary() table in statsmodels shows exactly **which predictors have positive/negative effects and whether they’re significant**.
- It’s closer to what you’d get in R or in academic/statistical software.

3. More statistical model types
- Beyond logistic regression: GLMs, survival models, mixed models, time-series (ARIMA), etc.
- You can use consistent syntax for all.

In [25]:
# Prepare features - encode categorical variables
df_model = df.copy()
# Convert categorical variable into dummy/indicator variables.
df_model = pd.get_dummies(df_model, columns=['gender', 'hometown', 'age_group'], drop_first=True)

# Define X and y
X = df_model.drop(columns=['user_id', 'action_flag'])
y = df_model['action_flag']

# Add intercept for _statsmodels_ logistic regression
# If you don’t add it manually, it will fit a model without one, which often gives biased
# coefficients unless your data is centered.
X_sm = sm.add_constant(X)

# Convert all boolean columns to integers
bool_cols = X_sm.select_dtypes(include=['bool']).columns
X_sm[bool_cols] = X_sm[bool_cols].astype(int)

# Fit logistic regression with statsmodels for interpretability
logit_model = sm.Logit(y, X_sm)
result = logit_model.fit()

print(result.summary())

Optimization terminated successfully.
         Current function value: 0.487362
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:            action_flag   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      992
Method:                           MLE   Df Model:                            7
Date:                Sun, 10 Aug 2025   Pseudo R-squ.:                 0.03140
Time:                        19:22:45   Log-Likelihood:                -487.36
converged:                       True   LL-Null:                       -503.16
Covariance Type:            nonrobust   LLR p-value:                 4.816e-05
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.1148      0.603     -0.190      0.849      -1.297       1.068
age           

In [31]:
# Extract significant feature names, excluding 'const' explicitly if present
significant_feats = result.pvalues[result.pvalues < 0.05].index.tolist()

if 'const' in significant_feats:
    significant_feats.remove('const')

print("\nSignificant features:", significant_feats)

# Now select only those features from your original X DataFrame
# YOU COULD use this for building a model using sklearn and use only the significant features
X_selected = X[significant_feats]


Significant features: ['gender_Male']


In [7]:
df_model.head()

,user_id,age,action_flag,gender_Male,hometown_TownB,hometown_TownC,age_group_26-35,age_group_36-45,age_group_46-60
0,1,29,0,True,True,False,True,False,False
1,2,33,1,False,True,False,True,False,False
2,3,41,0,False,False,False,False,True,False
3,4,36,0,False,False,False,False,True,False
4,5,25,0,True,False,True,False,False,False


### Interpretation
* **Coefficients show how each feature increases or decreases the log odds of performing the action.**
* For example, a positive coefficient for gender_Female means females are more likely to perform the action.
* P-values show statistical significance of each predictor.

## Model Evaluation & Feature Importance (scikit-learn)

In [7]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Logistic regression with sklearn
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Predict and classification report
y_pred = clf.predict(X_test)
print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=0)) # zero_division: sklearn throws the warning and sets precision to 0.0 by default.

# Feature importance (coefficients)
feature_importance = pd.Series(clf.coef_[0], index=X.columns).sort_values(ascending=False)
print("\nFeature Importance (logistic regression coefficients):\n", feature_importance)


Classification Report:
               precision    recall  f1-score   support

           0       0.80      1.00      0.89       200
           1       0.00      0.00      0.00        50

    accuracy                           0.80       250
   macro avg       0.40      0.50      0.44       250
weighted avg       0.64      0.80      0.71       250


Feature Importance (logistic regression coefficients):
 hometown_TownB     0.250804
age_group_46-60    0.179527
hometown_TownC     0.167558
age_group_36-45    0.155321
age_group_26-35    0.129470
age               -0.019596
gender_Male       -0.623933
dtype: float64


## Why using statsmodels and logistic regression combined?

| Aspect                     | statsmodels                              | scikit-learn                         |
|----------------------------|-----------------------------------------|------------------------------------|
| **Purpose**                | Statistical inference, interpretability | Predictive modeling, performance   |
| **Output**                 | Coefficients, p-values, confidence intervals | Predictions, accuracy metrics, feature importances (coefficients without p-values) |
| **Use case in your code**  | Understand which variables are significant and their effect sizes | Evaluate model performance on unseen data; make predictions |
| **Model evaluation**       | Doesn’t have built-in train/test splitting or accuracy metrics | Designed for pipeline: split data, train, test, score |
| **Interpretation**         | Easy to communicate statistical significance | Provides classification reports, ROC, etc. for practical performance |


**So why both?**
- **Statsmodels** gives you the “scientific explanation”: Which features truly matter? How confident are we?
- **scikit-learn** gives you the “machine learning practicality”: How well does the model perform on new/unseen data? What’s the precision, recall, F1?

### Typical workflow:
1. **Use statsmodels** for exploratory data analysis, model building, and to check which variables are important and statistically significant.
2.	Once you settle on features, **switch to scikit-learn for building a robust predictive pipeline**, hyperparameter tuning, cross-validation, and performance evaluation.
3.	Use scikit-learn models to generate predictions, score, and integrate into production systems.

# Advanced

## Interaction Terms (Gender × Age)

Adding interaction terms lets you see if the effect of one feature depends on another (e.g., does gender effect vary by age?).

How to do it:
* In logistic regression, add a new feature that’s the product of gender_Female × age (or age_group).
* Check if the interaction coefficient is significant.

In [11]:
df_model.head()

,user_id,age,action_flag,gender_Male,hometown_TownB,hometown_TownC,age_group_26-35,age_group_36-45,age_group_46-60,genderAge_interaction
0,1,29,0,1,1,0,1,0,0,29
1,2,33,1,0,1,0,1,0,0,0
2,3,41,0,0,0,0,0,1,0,0
3,4,36,0,0,0,0,0,1,0,0
4,5,25,0,1,0,1,0,0,0,25


In [10]:
# Convert all boolean columns to integers
bool_cols = df_model.select_dtypes(include=['bool']).columns
df_model[bool_cols] = df_model[bool_cols].astype(int)

In [32]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import statsmodels.api as sm
from lifelines import KaplanMeierFitter
from sklift.models import TwoModels

# --------------------
# 1. Synthetic dataset
# --------------------
np.random.seed(42)
n = 1000
df = pd.DataFrame({
    'user_id': range(1, n+1),
    'gender': np.random.choice(['Male', 'Female'], n),
    'age': np.random.randint(18, 60, n),
    'hometown': np.random.choice(['TownA', 'TownB', 'TownC'], n),
})

# Action flag influenced by gender & age
df['action_flag'] = np.random.binomial(
    1,
    p=0.1 + 0.1*(df['gender'] == 'Female') + 0.002*(60 - df['age']),
    size=n
)

# -------------------------
# Add signup/action dates
# -------------------------
df['signup_date'] = pd.Timestamp('2023-01-01') + pd.to_timedelta(
    np.random.randint(0, 30, n), unit='D'
)

# Action date: only for those with action_flag=1, else NaT
df['action_date'] = pd.NaT
mask = df['action_flag'] == 1
df.loc[mask, 'action_date'] = df.loc[mask, 'signup_date'] + pd.to_timedelta(
    np.random.randint(1, 60, mask.sum()), unit='D'
)

## Interaction

**What is it?**
An interaction term in regression models **captures the effect of two or more features working together on the outcome** — beyond their individual effects.
Instead of assuming each variable affects the outcome independently and additively, an interaction term tests if the effect of one variable depends on the level of another.

**Example:**
The effect of gender on buying behavior might be different for different age groups. So you include a term gender × age to capture that nuance.

**When to use it?**
- When you suspect that the impact of one feature varies depending on another.
- To capture complex relationships that simple additive models miss.
- When domain knowledge suggests variables interact.

**What it does for you:**
- Helps uncover conditional relationships between features and outcome.
- Improves model fit and interpretability if interactions matter.
- Allows more precise targeting (e.g., specific age-gender combos).

In [ ]:
# -------------------------
# 2. Interaction term
# -------------------------
df['gender_male'] = (df['gender'] == 'Male').astype(int)
df['interaction_gender_age'] = df['gender_male'] * df['age']

X_inter = df[['gender_male', 'age', 'interaction_gender_age']]
y_inter = df['action_flag']

X_inter = sm.add_constant(X_inter)
logit_inter = sm.Logit(y_inter, X_inter)
result_inter = logit_inter.fit()
print("\n=== Logistic Regression with Interaction Term ===")
print(result_inter.summary())

## Survival analysis

**What is it?**

Survival analysis **models time until an event happens** — e.g., time to churn, time to first purchase, time to feature adoption.
It deals with censored data (users who haven’t yet taken the event by the observation end).

**When to use it?**
- When you want to **understand how long users take to perform an action or experience an event**.
- When users might not have done the event yet (censoring).
- When comparing time-to-event distributions between groups (e.g., gender, treatment vs control).

**What it does for you:**
- Estimates probabilities of surviving (not acting) past a time point — e.g., 30-day retention rates.
- Enables comparison of time-to-event curves across segments or treatments.
- Can model effects of covariates on timing using Cox proportional hazards or other models.

In [33]:
# -------------------------
# 3. Survival analysis
# -------------------------
df['time_to_action'] = (df['action_date'] - df['signup_date']).dt.days
# Fill NaN with censoring limit (e.g., 60 days)
df['time_to_action'] = df['time_to_action'].fillna(60)
df['event_observed'] = df['action_flag']

kmf = KaplanMeierFitter()
kmf.fit(durations=df['time_to_action'], event_observed=df['event_observed'])
print("\n=== Survival Analysis (first 10 survival probabilities) ===")
print(kmf.survival_function_.head(10))


=== Survival Analysis (first 10 survival probabilities) ===
          KM_estimate
timeline             
0.0             1.000
1.0             0.999
2.0             0.996
3.0             0.994
4.0             0.993
5.0             0.992
7.0             0.990
8.0             0.985
9.0             0.981
10.0            0.978


This output is the **Kaplan–Meier survival curve estimate** showing the probability of not having performed the event (e.g., an action) up to each time point (in days).

| timeline (days) | KM_estimate (survival probability)          | Interpretation                                          |
|-----------------|---------------------------------------------|---------------------------------------------------------|
| 0.0             | 1.000 (or 100%)                             | At day 0, everyone has *not* acted yet (starting point).|
| 1.0             | 0.999 (99.9%)                              | About 99.9% of users haven’t performed the action by day 1.|
| 2.0             | 0.996 (99.6%)                              | About 99.6% haven’t acted by day 2, so 0.4% did act between day 1 and 2.|
| 10.0            | 0.978 (97.8%)                              | By day 10, about 2.2% of users have performed the action.|

## Uplift Modeling

**What is it?**

Uplift modeling **predicts the incremental impact of a treatment or action on individual outcomes**. It’s sometimes called causal modeling or heterogeneous treatment effect modeling.

**When to use it?**
- When you run experiments or campaigns and want to identify who truly benefits from the treatment.
- To target interventions more effectively by focusing on users with positive uplift.
- When you want to maximize ROI by avoiding “spenders” who would buy anyway (no uplift).

**What it does for you:**
- Predicts who will respond positively to treatment vs control.
- Helps avoid wasting resources on “doomed” or “sure thing” customers.
- Enables personalized treatment assignment.

In [34]:
# -------------------------
# 4. Uplift modeling
# -------------------------
# Simulate treatment assignment
df['treatment'] = np.random.binomial(1, 0.5, n)

# For demo: outcome is more likely if treated and female
df['outcome'] = np.random.binomial(
    1,
    0.05 + 0.15*df['treatment'] + 0.10*(df['gender'] == 'Female'),
    size=n
)

X_uplift = pd.get_dummies(df[['gender', 'age', 'hometown']], drop_first=True)
y_uplift = df['outcome']
treatment = df['treatment']

uplift_model = TwoModels(
    estimator_trmnt=LogisticRegression(max_iter=500),
    estimator_ctrl=LogisticRegression(max_iter=500),
    method='vanilla'
)
uplift_model.fit(X_uplift, y_uplift, treatment)

uplift_scores = uplift_model.predict(X_uplift)
print("\n=== Uplift Scores (first 10) ===")
print(uplift_scores[:10])


=== Uplift Scores (first 10) ===
[0.14520826 0.08809447 0.19395015 0.16810701 0.15521849 0.16832896
 0.1283236  0.10356832 0.19814396 0.02370063]


Those uplift scores represent, for each user, the predicted incremental impact (or lift) of the treatment on the likelihood of the desired outcome compared to no treatment.

**What does this imply practically?**
- Higher uplift scores = better candidates for treatment.
- You can prioritize targeting users with the highest uplift to maximize campaign ROI.
- Users with low or negative uplift might be left untreated to save costs or avoid negative effects.

How to use uplift scores?
- Sort users by their uplift score descending.
- Target top X% (depending on budget).
- Monitor real-world results to validate the model’s predictions.

|Score Example | Meaning|
|--------------|--------|
|0.19 (19%)|The model predicts this user is about 19 percentage points more likely to take the action because of the treatment.|
|0.02 (2%)|The treatment might increase the chance by 2 percentage points—small but positive.|
|Negative values (if any)|The treatment might actually decrease the chance for this user (harmful or no effect).|



### Summary:
1. Interaction term → Adds gender × age to the logistic regression model and prints the coefficients.
2.	Survival analysis → Creates fake signup/action dates, computes time_to_action, handles censoring for users without an action, and prints a Kaplan–Meier survival table.
3.	Uplift modeling → Simulates treatment assignment and outcome, fits a treatment/control model, and outputs predicted uplift scores.

| Extension Topic   | Method / Tool                        | Input Data Needed                 | What it Adds                                  |
|-------------------|--------------------------------------|------------------------------------|-----------------------------------------------|
| Interaction Terms | Logistic regression with product terms| User features                      | Detect feature interplay, nuanced segments   |
| Survival Analysis | Kaplan-Meier, Cox model              | Time-to-action + censoring         | Model time dimension of user behavior        |
| Uplift Modeling   | Two-model approach, uplift forests   | Treatment assignment + outcome     | Identify who benefits most from campaigns    |